# Monte Carlo

In [ ]:
import numpy
from matplotlib import pyplot
%matplotlib inline

## Random numbers

### Built-in

In [ ]:
x = numpy.random.normal(loc=0, scale=1, size=100000)

# Plot
pyplot.xlim(-5, 5)

binwidth = 0.2
bins = numpy.arange(-5, 5, binwidth)
pyplot.hist(x, bins, normed=True);

#Expected
bin_centers = bins[:-1] + binwidth/2
pyplot.plot(bin_centers, numpy.exp(-bin_centers**2/2.0)/numpy.sqrt(2*numpy.pi))

In [ ]:
x = numpy.random.exponential(scale=4, size=100000)

# Plot
pyplot.xlim(0, 20)

binwidth = 0.5
bins = numpy.arange(0, 20, binwidth)
pyplot.hist(x, bins, normed=True);

#Expected
bin_centers = bins[:-1] + binwidth/2
pyplot.plot(bin_centers, (1/4)*numpy.exp(-(1/4)*bin_centers))

### Box-Muller

In [ ]:
def gauss(avg, stdev):
    # Implémentation de la méthode Box-Muller polaire
    # Cette forme est plus rapide et plus précise autour des 0 des nombres aléatoires

    s = 2.0
    while ( (s == 0) or ( s>=1 ) ):
            i = numpy.random.random_sample(2)
            i[:] = i[:]*2 - 1
            s = sum(i*i)

    return avg + stdev * i[:]*numpy.sqrt((-2*numpy.log(s))/s)

x = numpy.empty(100000)
for i in range(0, 50000):
    t = gauss(0, 1)
    x[i] = t[0]
    x[50000 + i] = t[1]

# Plot
pyplot.xlim(-5, 5)

binwidth = 0.2
bins = numpy.arange(-5, 5, binwidth)
pyplot.hist(x, bins, normed=True);

#Expected
bin_centers = bins[:-1] + binwidth/2
pyplot.plot(bin_centers, numpy.exp(-bin_centers**2/2.0)/numpy.sqrt(2*numpy.pi))

## Brownian motion

### Systematic

$$ dx=cdt+\alpha \sqrt{dt}\cal{N(0,1)} $$

In [ ]:
npart = 100000

T = 50.0
nt = 500
dt = T/nt

alpha = 2.0
c = 0.5

# Generate a set of initial positions based on the Gaussian distribution
x = numpy.random.normal(loc=0,scale=1,size=npart)

# Evolution
for i in range(nt):
    xi = x.copy()
    x = xi + c*dt + alpha*numpy.sqrt(dt)*numpy.random.normal(0, 1, npart)

# Plot
pyplot.figure(figsize=(15, 10))
pyplot.xlim(-50, 100)

binwidth = 0.75
bins = numpy.arange(-50, 100, binwidth)
pyplot.hist(x, bins, normed=True);

#Expected
bin_centers = bins[:-1] + binwidth/2
pyplot.plot(bin_centers, numpy.exp(-(bin_centers - c*T)**2/(2.0*alpha**2*T)) \
                            / numpy.sqrt(2*numpy.pi*alpha**2*T))

### Friction

$$ dx=-\gamma x dt +\alpha \sqrt{dt}\cal{N(0,1)} $$

In [ ]:
npart = 100000

T = 50.0
nt = 500
dt = T/nt

alpha = 2.0
gamma = 0.5

# Generate a set of initial positions based on the Gaussian distribution
x = numpy.random.normal(loc=0, scale=1, size=npart)

# Evolution
for i in range(nt):
    xi = x.copy()
    x = xi - gamma*xi*dt + alpha*numpy.sqrt(dt)*numpy.random.normal(0, 1, npart)

# Plot
pyplot.figure(figsize=(15, 10))
pyplot.xlim(-50, 50)

binwidth = 0.5
bins = numpy.arange(-50, 50, binwidth)
pyplot.hist(x, bins, normed=True);

#Expected
bin_centers = bins[:-1] + binwidth/2
pyplot.plot(bin_centers, numpy.exp(-(bin_centers)**2/(2.0*alpha**2*T)) \
                            / numpy.sqrt(2*numpy.pi*alpha**2*T))

## Ising model

### Functions definition

In [ ]:
def energy_array(sp, alpha):
    return -alpha*sp[:, :]* \
        (numpy.roll(sp, 1, axis=0) + numpy.roll(sp, -1, axis=0) + \
         numpy.roll(sp, 1, axis=1) + numpy.roll(sp, -1, axis=1))

def total_energy(sp, alpha):
    return energy_array(sp, alpha).sum()

In [ ]:
def total_magnetization(sp):
    return sp.sum()

In [ ]:
def metropolis_at_site(sp,alpha,ix,iy):
    """ Flips a dipole at site ix, iy when probability condition is met 
   
    Parameters:
    ----------
    sp: numpy array
        array of spins
    alpha  : real
        coupling constant J/(kb*T)
    ix   : int
        location in x
    iy   : int
        location in y
    """
    sigma=sp[ix,iy]
    energy_before_flip = energy_at_site(sp,alpha,sigma,ix,iy)
    sigma = -sigma
    energy_if_site_flipped = energy_at_site(sp,alpha,sigma,ix,iy)
    
    # Flip the site with Metropolis probability
    # Condition is always satisifed if dE < 0
    if (numpy.random.random_sample()<numpy.exp(-(energy_if_site_flipped \
                                               -energy_before_flip))):
        sp[ix,iy]=-sp[ix,iy]

def ising_model_metropolis(sp, NMC, nx, ny, alpha):
    """ Creates a sequence of states for the Ising model using
    the Metropolis algorithm
   
    Parameters:
    ----------
    sp   : initial lattice state
    nx   : int
        Discretization points in x
    ny   : int
        Discretization points in y
    NMC  : int
        Number of states to create
    alpha  : real
        coupling constant J/(kb*T)
    Returns:
    -------
    states: sequence of states
    """
    states = numpy.empty([NMC+1,nx,ny])
    states[0] = sp.copy()
    
    for i in range(1,NMC+1):
        for j in range(0,nx*ny):
            ix=numpy.random.random_integers(0,nx-1)
            iy=numpy.random.random_integers(0,ny-1)
            metropolis_at_site(sp,alpha,ix,iy)
        states[i]=sp.copy()
    return states

In [ ]:
NMC = 100

nx = 100
ny = 100

na = 20

alpha = numpy.linspace(0.25, 0.55, na)
#alpha = [0.1, 0.2, 0.3, 0.35, 0.37, 0.39, 0.41, 0.43, 0.45, 0.5, 0.6, 0.7]
energy = numpy.zeros(na)
magn = numpy.zeros(na)

for i in range(0, na):
    sp = numpy.ones([nx,ny])
    states = ising_model_metropolis(sp, NMC, nx, ny, alpha[i])
    for j in range(0, NMC+1):
        energy[i] += total_energy(states[j], alpha[i])/(NMC+1)
        magn[i] += total_magnetization(states[j])/(NMC+1)

In [ ]:
pyplot.figure(figsize=(15, 10))
pyplot.plot(alpha, energy)
pyplot.plot(alpha, magn)